In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import warnings
warnings.filterwarnings("ignore")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
# import data
dataset = pd.read_csv('hotel_bookings.csv')

# Data prepatation
# do not require
data = dataset.drop(['arrival_date_year', 'reservation_status', 
                  'reservation_status_date','company'], axis=1)
data

,hotel,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,Resort Hotel,0,342,July,27,1,0,0,2,0.0,...,C,C,3,No Deposit,NaN,0,Transient,0.00,0,0
1,Resort Hotel,0,737,July,27,1,0,0,2,0.0,...,C,C,4,No Deposit,NaN,0,Transient,0.00,0,0
2,Resort Hotel,0,7,July,27,1,0,1,1,0.0,...,A,C,0,No Deposit,NaN,0,Transient,75.00,0,0
3,Resort Hotel,0,13,July,27,1,0,1,1,0.0,...,A,A,0,No Deposit,304.0,0,Transient,75.00,0,0
4,Resort Hotel,0,14,July,27,1,0,2,2,0.0,...,A,A,0,No Deposit,240.0,0,Transient,98.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,August,35,30,2,5,2,0.0,...,A,A,0,No Deposit,394.0,0,Transient,96.14,0,0
119386,City Hotel,0,102,August,35,31,2,5,3,0.0,...,E,E,0,No Deposit,9.0,0,Transient,225.43,0,2
119387,City Hotel,0,34,August,35,31,2,5,2,0.0,...,D,D,0,No Deposit,9.0,0,Transient,157.71,0,4
119388,City Hotel,0,109,August,35,31,2,5,2,0.0,...,A,A,0,No Deposit,89.0,0,Transient,104.40,0,0


We have 28 features, which gives us information about a customer's hotel booking and 119,390 observations, which are the number of bookings that either had or had not churned.

In [3]:
# idfentify which columns must be encoded with dummy variables
variables_to_dummy = []
for col in data.columns:
    if data[col].dtypes != 'int64':
        if data[col].dtypes != 'float64':
            variables_to_dummy.append(col)

# identify if na values
data.isna().sum()

hotel                                 0
is_canceled                           0
lead_time                             0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              4
babies                                0
meal                                  0
country                             488
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                             16340
days_in_waiting_list                  0
customer_type                         0


In [4]:
data.groupby('children').size()

children
0.0     110796
1.0       4861
2.0       3652
3.0         76
10.0         1
dtype: int64

In [5]:
# The number children is not even or normally distribution 
# Therefore we will use the median to replce the na values

child_med = data['children'].median() 
data['children'] = data['children'].fillna(value=child_med)

In [6]:
# The values of the agent variable refers to a travel agency's ID
#  that made the booking. Therefore we cannot take the median or mean
#  We will input 0 for no agency.

data['agent'] = data['agent'].fillna(value=0)

data.isna().sum() # dummy variables will take care of the na values in country

hotel                               0
is_canceled                         0
lead_time                           0
arrival_date_month                  0
arrival_date_week_number            0
arrival_date_day_of_month           0
stays_in_weekend_nights             0
stays_in_week_nights                0
adults                              0
children                            0
babies                              0
meal                                0
country                           488
market_segment                      0
distribution_channel                0
is_repeated_guest                   0
previous_cancellations              0
previous_bookings_not_canceled      0
reserved_room_type                  0
assigned_room_type                  0
booking_changes                     0
deposit_type                        0
agent                               0
days_in_waiting_list                0
customer_type                       0
adr                                 0
required_car

In [7]:
variables_to_dummy = []
for col in data.columns:
    if data[col].dtypes == 'object':
        variables_to_dummy.append(col)


for var in variables_to_dummy:
    data[var] = pd.get_dummies(data[var])

In [8]:
for col in data.columns:
    for entry in data[col]:
        if entry < 0:
            print(entry, " in column", col) 
            # we see there is indeed a negative value in adr

-6.38  in column adr


adr stands for Average Daily Rate, which is defined by dividing the sum of all lodging transactions by the total number of staying nights. Therefore this must have been an error either when inputing the data or when the adr was calculated. Therefore we will assume the numerical value is correct and make it positive.

In [9]:
data['adr'] = data['adr'].abs()
any(data['adr'] < 0)

False

In [10]:
from sklearn.model_selection import train_test_split

X = data.drop(['is_canceled'], axis=1)
Y = data['is_canceled']
print(X.shape)
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.50, random_state=66)

(119390, 27)
(119390,)


## Logistic Model

In [11]:
# Logistic Regression Model
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression().fit(x_train, y_train)

log_predict = log_model.predict(x_test)

array([0, 0, 1, 1, 1, 0, 1, 1, 1, 0])

In [12]:
from sklearn import metrics

log_accuracy = metrics.accuracy_score(y_test, log_predict)
log_precision = metrics.precision_score(y_test, log_predict)

print("Model's Accuracy: {}  Model's Precision: {}".format(
    round(log_accuracy,2), round(log_precision,2)))

Model's Accuracy: 0.78  Model's Precision: 0.86


In [13]:
print(metrics.classification_report(y_test, log_predict))

              precision    recall  f1-score   support

           0       0.76      0.95      0.84     37681
           1       0.86      0.48      0.61     22014

    accuracy                           0.78     59695
   macro avg       0.81      0.72      0.73     59695
weighted avg       0.80      0.78      0.76     59695



## Support Vector Machine

In [14]:
# Support Vector Machine Model

from sklearn.svm import LinearSVC

svm_model = LinearSVC(random_state = 100).fit(x_train, y_train)
svm_predict = svm_model.predict(x_test)

svm_accuracy = metrics.accuracy_score(y_test, svm_predict)
svm_precision = metrics.precision_score(y_test, svm_predict)

print("Model's Accuracy: {}  Model's Precision: {}".format(
    round(svm_accuracy,2), round(svm_precision,2)))


Model's Accuracy: 0.73  Model's Precision: 0.85


In [15]:
print(metrics.classification_report(y_test, svm_predict))

              precision    recall  f1-score   support

           0       0.71      0.97      0.82     37681
           1       0.85      0.31      0.45     22014

    accuracy                           0.73     59695
   macro avg       0.78      0.64      0.63     59695
weighted avg       0.76      0.73      0.68     59695



## Naive Bayes

In [16]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB().fit(x_train, y_train)
nb_predict = nb_model.predict(x_test)

nb_accuracy = metrics.accuracy_score(y_test, nb_predict)
nb_precision = metrics.precision_score(y_test, nb_predict)

print("Model's Accuracy: {}  Model's Precision: {}".format(
    round(nb_accuracy,2), round(nb_precision,2)))

Model's Accuracy: 0.64  Model's Precision: 0.51


In [17]:
print(metrics.classification_report(y_test, nb_predict))

              precision    recall  f1-score   support

           0       0.73      0.69      0.71     37681
           1       0.51      0.56      0.53     22014

    accuracy                           0.64     59695
   macro avg       0.62      0.62      0.62     59695
weighted avg       0.65      0.64      0.64     59695



The naive bayes model has the least accuracy and precision. This is due to the underlying assumption that each feature is independent, hence is naive. 

## Conclusion

We will prefer to perform the the logistic regression. We were able to predict the cancellation of a booking with 78% accuracy and 86% precision.



In [209]:
# random test sample
c = np.empty(10)
random_sample = pd.DataFrame(np.random.randn(10,27))
for i in range(10000):
    c += log_model.predict(random_sample)
c = pd.DataFrame(np.round(c/10000))
c

,0
0,0.0
1,1.0
2,1.0
3,0.0
4,1.0
5,0.0
6,1.0
7,0.0
8,0.0
9,1.0


We found 10,000 and found the average outcome for each observation.

Therefore the random sample has the above outcomes 

In [210]:
random_sample['Outcomes'] = c
random_sample

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,Outcomes
0,0.408156,-0.555132,-0.371549,-1.827303,0.808223,0.282140,-0.874811,-1.627365,0.654091,-1.337989,...,-0.513379,1.002216,2.572210,0.605107,-1.089903,0.253579,0.847374,-1.405714,-0.509240,0.0
1,0.531326,0.713196,0.921482,2.081292,-0.146656,-1.454227,1.406788,0.809487,-1.078386,0.880571,...,0.193378,1.326147,-0.643381,1.990825,-0.528608,0.113404,1.173631,-0.778644,-1.603451,1.0
2,-1.006406,-0.132274,-0.997935,0.797763,1.411655,-0.133754,0.582470,-0.114707,-0.944640,1.137764,...,0.272097,0.242357,-0.541490,-0.602432,0.843995,0.609787,0.146392,-1.009675,-0.896952,1.0
3,-1.271607,-1.213563,-2.139089,0.285955,0.784727,-0.062598,-0.859167,0.612893,-0.220309,0.183175,...,-0.984921,-0.569669,-1.149170,-0.183039,1.390587,-0.602758,-0.374805,2.495581,-0.509908,0.0
4,-1.066032,0.459591,-1.043282,0.737196,1.369338,0.696137,-1.239360,-0.595851,-0.075371,-0.837649,...,0.455583,1.632183,0.847341,-1.060586,0.878434,-0.667193,1.051715,-0.597127,-1.622360,1.0
5,-0.869975,0.034308,0.829685,-0.718222,1.213580,1.751866,1.779569,-1.304158,0.155278,-0.418287,...,-0.327335,1.654421,-0.521006,0.490708,0.621872,0.566354,-0.338104,0.452194,0.641809,0.0
6,-0.125087,0.104437,-0.048904,0.516975,0.462408,-1.125127,-0.534253,0.010603,0.596161,-1.513604,...,0.085220,1.476106,0.338470,1.252064,0.870231,-0.022431,0.309527,-1.055250,-0.604431,1.0
7,0.807902,-0.898651,-0.038807,-0.870602,-0.521710,0.099370,0.164449,0.939354,0.172171,0.447116,...,-1.410227,0.710146,0.825928,-0.603666,-0.467690,-1.621925,2.406816,0.338759,-1.309692,0.0
8,1.292855,0.990103,-0.316984,0.285808,-0.139153,0.072751,-2.230006,0.993109,-1.373289,-2.723281,...,0.335125,-1.434543,0.499539,0.652358,-1.000335,-0.142981,1.238447,0.076258,1.327503,0.0
9,-0.400693,-0.166310,-0.733023,0.323485,0.669919,1.757180,-1.312581,0.795530,2.024860,0.759132,...,2.852890,-0.182541,-0.953358,0.059749,0.116698,-0.695335,-0.431613,1.073313,0.960113,1.0
